In [1]:
import numpy as np
import pandas as pd
import pickle
from collections import defaultdict

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import *
from sklearn.model_selection import *
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.preprocessing import LabelEncoder, PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.utils import shuffle

from sklearn.dummy import DummyRegressor
from sklearn.linear_model import *
from sklearn.neighbors import KNeighborsRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import StackingRegressor, VotingRegressor, RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor, XGBClassifier

from hyperopt import fmin, tpe, hp, Trials

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-whitegrid')
sns.set_context(font_scale=1.3)

In [2]:
df = pd.read_csv('Data/reproducibility.csv').sample(frac=1, random_state=1).reset_index(drop=True)
df.head()

,peptide,peptide_aromaticity,peptide_gravy,peptide_helix,peptide_instability_index,peptide_isoelectric_point,peptide_len,peptide_log_len,peptide_log_weight,peptide_mean,...,protein_len,protein_log_len,protein_log_weight,protein_molar_extinction_coefficient,protein_sheet,protein_turn,protein_weight,reproducibility,reproducibility_corrected,peptide_detectability
0,SELTQQLNALFQDK,0.071429,-0.742857,0.285714,39.107143,4.368781,14,2.639057,7.399267,21.571060,...,480,6.173786,10.925809,14900.0,0.372917,0.162500,55592.8205,0.134116,-0.416929,0.863536
1,EETGQVLER,0.000000,-1.288889,0.222222,27.300000,4.252773,9,2.197225,6.966134,23.123484,...,613,6.418365,11.114425,114625.0,0.225122,0.295269,67132.6103,0.118270,-0.276561,0.539523
2,QLYGDTGVLGR,0.090909,-0.263636,0.363636,-20.009091,5.835682,11,2.397895,7.071824,22.152872,...,184,5.214936,9.929104,22982.5,0.288043,0.163043,20518.9427,0.730757,-0.019208,0.858968
3,EVQVFEITENSAK,0.076923,-0.415385,0.307692,33.061538,4.252773,13,2.564949,7.308954,18.674903,...,188,5.236442,9.948520,4470.0,0.218085,0.202128,20921.2417,0.245870,0.106907,0.876252
4,EDYICYAR,0.250000,-0.662500,0.375000,62.425000,4.370430,8,2.079442,6.939377,23.851474,...,127,4.844187,9.523820,18450.0,0.188976,0.377953,13681.7806,0.768097,0.377618,0.649913


In [3]:
class ResultsContainer:
    def __init__(self, path=None):
        self.container = defaultdict(
            lambda: defaultdict(
                lambda: dict(
                    optim_history=[],
                    params=defaultdict(list),
                    results=defaultdict(list),
                )
            )
        )
        if path is not None:
            self.load(path)

    def update_from_list(self, exp_name, optim_history, params, results):
        for model_name in optim_history:
            self.update(exp_name, model_name,
                        optim_history[model_name],
                        params[model_name],
                        results[model_name])

    def update(self, exp_name, model_name, optim_history, params, results):
        self.container[exp_name][model_name]['optim_history'].append(optim_history)
        for k, v in params.items():
            self.container[exp_name][model_name]['params'][k].append(v)
        for k, v in results.items():
            self.container[exp_name][model_name]['results'][k].append(v)
        
    def save(self, path):
        pickle.dump(self.to_dict(), open(path, 'wb'))
        
    def load(self, path):
        self.container.update(pickle.load(open(path, 'rb')))
        
    def update_file(self, path):
        container = ResultsContainer(path)
        container_new = self.container
        for exp_name, models in container_new.items():
            for model_name, model_container in models.items():
                container[exp_name][model_name] = model_container
        container.save(path)

    def to_dict(self):
        return self.rec_to_dict(self.container)
    
    def rec_to_dict(self, d):
        if type(d)!=defaultdict and type(d)!=dict:
            return d
        for k, v in d.items():
            if type(v)==defaultdict or type(v)==dict:
                d[k] = self.rec_to_dict(v)
        return dict(d)
    
    def __getitem__(self, key):
        return self.container[key]
    

def get_data(df, features=[]):
    xs = {}
    encoder = CountVectorizer(analyzer='char', lowercase=False)
    xs['Counts'] = encoder.fit_transform(df.peptide.values).toarray()
    xs['Binary'] = np.where(xs['Counts']>0, 1, 0)
    xs['Relative'] = xs['Counts'] / xs['Counts'].sum(1, keepdims=True)
    
    embeds_path = '../Data/Embeds/'
    for name, file in (('Doc2Vec', 'from_model_3_1.pkl'), ('Bert', 'bert_embeds.pkl')):
        embeds = df.peptide.map(pickle.load(open(embeds_path+file, 'rb'))).values
        xs[name] = np.vstack(embeds)
    
    if features:
        x_meta = df[features].values.reshape(-1, len(features))
        for key, x in xs.items():
            xs[key] = np.hstack([x, x_meta])
        xs['Meta'] = x_meta

    y = df.reproducibility.values
    groups = LabelEncoder().fit_transform(df['protein_id'].values)
    features =  list(encoder.get_feature_names()) + features
    return xs, y, groups, features 

def hyperopt(xs, y, Model, space, evals=100, cv_params={}, params={}, int_params={}, Scaler=None):
    def objective(space):
        for key, value in space.items():
            if key != 'features':
                if key in int_params:
                    params[key] = int(value)
                else:
                    params[key] = value
        
        if Scaler is None:
            model = Model(**params)
        else:
            model = make_pipeline(Scaler(), Model(**params))
            
        if 'features' in space:
            x = xs[space['features']]
        else:
            x = xs

        scores = -cross_val_score(model, x, y, **cv_params)
        return scores.mean() 
    
    params = params.copy()
    int_params = int_params.copy()
    default_params = params.copy()
    trials = Trials()
    best = fmin(fn=objective,
                space=space,
                algo=tpe.suggest,
                max_evals=evals,
                trials=trials)

    history = defaultdict(list)
    for t in trials:
        history['loss'].append(t['result']['loss'])
        for k, v in default_params.items():
            history[k].append(v)
        for k, v in t['misc']['vals'].items():
            if k in int_params:
                history[k].append(int(v[0]))
            else:
                history[k].append(v[0])

    for k, v in best.items():
        if k in int_params:
            params[k] = int(v)
        else:
            params[k] = v
    
    return params, dict(history)

def optimize_models(x, y, model_optim_params, cv_params, evals=1):
    optim_params, optim_history = {}, {}
    for model_name, model_params in model_optim_params.items():
        params, history = hyperopt(x, y, **model_params,
                                   cv_params=cv_params, evals=evals)
        optim_params[model_name] = params
        optim_history[model_name] = history
    return optim_params, optim_history

def get_model(Model, optim_params, optim_results):
    model_name = Model.__name__
    if model_name in optim_results:
        if model_name == 'XGBRegressor':
            optim_results[model_name]['random_state'] = np.random.randint(1000)
        if model_name in optim_params:
            model = optim_params[model_name]['Model'](**optim_results[model_name])
        else:
            model = Model(**optim_results[model_name])
        if 'Scaler' in optim_params[model_name]:
            scaler = optim_params[model_name]['Scaler']()
            return model_name, make_pipeline(scaler, model)
        else:
            return model_name, model
    else:
        return model_name, Model()

def compare_models(x_train, y_train, x_test, y_test,
                   model_classes, optim_params, optim_results):
    metrics = dict(
        MSE=mean_squared_error,
        MAE=mean_absolute_error,
        R2=r2_score,
        EV=explained_variance_score
    )
    results = defaultdict(dict)
    for Model in model_classes:
        model_name, model = get_model(Model, optim_params, optim_results)
        preds_train = model.fit(x_train, y_train).predict(x_train)
        preds_test = model.predict(x_test)
        for metric, func in metrics.items():
            results[model_name]['Train '+metric] = func(y_train, preds_train)
            results[model_name]['Test '+metric] = func(y_test, preds_test)
    return dict(results)

def print_table(exp_names, comp_results, ci=False, precision=5):
    model_names = comp_results[exp_names[0]]['Model']
    lines = []
    lines.append('\\begin{table}[h]')
    lines.append('\\centering')
    lines.append('\\begin{tabular}{l' + '|l'*len(exp_names) + '}')
    lines.append('\\textbf{Model}' + ''.join([' & \\textbf{'+e+'}' for e in exp_names]) + ' \\\\ \\hline')
    lines += [model for model in model_names]
    for exp_name in exp_names:
        for i in range(len(model_names)):
            mse = comp_results[exp_name]['Test MSE'][i]
            if ci:
                std = comp_results[exp_name]['Test Std'][i]
                text = str(np.round(mse, precision)) + ' \\pm ' + str(np.round(std, precision))
            else:
                text = str(np.round(mse, precision))
            text = ' & ' + text
            lines[i+4] += text
    for i in range(len(model_names)-1):
        lines[i+4] += ' \\\\ \\hline'
    lines.append('\\end{tabular}')
    lines.append('\\caption{}')
    lines.append('\\label{}')
    lines.append('\\end{table}')
    lines = '\n'.join(lines)
    print(lines)

In [9]:
# Define optimization parameter space
optim_params = {}
optim_params['KNeighborsRegressor'] = dict(
    Model = KNeighborsRegressor,
    space = {
        'n_neighbors': hp.qloguniform('n_neighbors', np.log(1), np.log(300), 1),
    },
    int_params={'n_neighbors'},
    Scaler=StandardScaler,
)
optim_params['KernelRidge'] =  dict(
    Model = KernelRidge,
    space = {
    'alpha': hp.loguniform('alpha', np.log(0.1), np.log(100)),
    'degree': hp.quniform('degree', 1, 4, 1),
    },
    params = {'kernel': 'poly'},
    int_params={'degree'},
    Scaler=StandardScaler,
)
optim_params['Ridge'] =  dict(
    Model = lambda degree, alpha: make_pipeline(
        PolynomialFeatures(degree, include_bias=False),
        Ridge(alpha=alpha)
    ),
    space = {
        'alpha': hp.loguniform('alpha', np.log(1e2), np.log(1e5)),
        'degree': hp.quniform('degree', 1, 3, 1),
    },
    int_params={'degree'},
    Scaler=StandardScaler,
)
optim_params['ElasticNet'] =  dict(
    Model = lambda degree, alpha, l1_ratio: make_pipeline(
        PolynomialFeatures(degree, include_bias=False),
        ElasticNet(alpha=alpha, l1_ratio=l1_ratio)
    ),
    space = {
        'alpha': hp.loguniform('alpha', np.log(1e-4), np.log(10)),
        'l1_ratio': hp.loguniform('l1_ratio', np.log(1e-4), np.log(1)),
        'degree': hp.quniform('degree', 1, 3, 1),
    },
    int_params={'degree'},
    Scaler=StandardScaler,
)
optim_params['RandomForestRegressor'] = dict(
    Model=RandomForestRegressor,
    space={
        'ccp_alpha': hp.loguniform('ccp_alpha', np.log(1e-6), np.log(1e-3)),
        'max_features': hp.uniform('max_features', 0.1, 1),
    },
    params = {'n_estimators': 300},
    int_params = {'max_depth', 'min_samples_leaf', 'n_estimators'},
)
optim_params['XGBRegressor'] = dict(
    Model=XGBRegressor,
    space={
        'learning_rate': hp.loguniform('learning_rate', np.log(0.001), np.log(0.3)),
        'colsample_bytree': hp.uniform ('colsample_bytree', 0.1, 1),
        'colsample_bylevel': hp.uniform ('colsample_bylevel', 0.1, 1),
        'subsample': hp.uniform ('subsample', 0.3, 1),
        'max_depth': hp.quniform('max_depth', 2, 8, 1),
#         'n_estimators': hp.qloguniform('n_estimators', np.log(100), np.log(1000), 10),
        'reg_lambda': hp.loguniform('reg_lambda', np.log(1e-4), np.log(10)),
        'reg_alpha': hp.loguniform('reg_alpha', np.log(1e-4), np.log(10)),
    },
    params = {'n_estimators': 300},
    int_params={'min_child_weight', 'max_depth', 'n_estimators'},
)
optim_params['MLPRegressor'] = dict(
    Model=MLPRegressor,
    space = {
        'hidden_layer_sizes': hp.qloguniform('hidden_layer_sizes', np.log(8), np.log(256), 2),
        'learning_rate_init': hp.loguniform('learning_rate_init', np.log(1e-3), np.log(1)),
        'alpha': hp.loguniform('alpha', np.log(1e-2), np.log(10)),
        'batch_size': hp.qloguniform('batch_size', np.log(8), np.log(256), 4),
    },
    params = {'solver':'sgd', 'learning_rate': 'adaptive'},
    int_params = {'hidden_layer_sizes', 'batch_size'},
    Scaler=StandardScaler,
)

# Define experiments
experiment_params = {}
# # SEQ
experiment_params['Counts'] = dict(
    encoding_type='Counts',
    features=[],
)
experiment_params['Relative'] = dict(
    encoding_type='Relative',
    features=[],
)
experiment_params['Binary'] = dict(
    encoding_type='Binary',
    features=[],
)
# SEQ + len
experiment_params['Counts+len'] = dict(
    encoding_type='Counts',
    features=['peptide_len'],
)
experiment_params['Relative+len'] = dict(
    encoding_type='Relative',
    features=['peptide_len'],
)
experiment_params['Binary+len'] = dict(
    encoding_type='Binary',
    features=['peptide_len'],
)
# Embeds
experiment_params['Doc2Vec'] = dict(
    encoding_type='Doc2Vec',
    features=[],
)
experiment_params['Bert'] = dict(
    encoding_type='Bert',
    features=[],
)
# Peptide
features = [
    'len', 'gravy', 'helix', 'turn', 'sheet', 'aromaticity',
    'instability_index', 'isoelectric_point', 'molar_extinction_coefficient'
]
features = ['peptide_' + f for f in features] + ['peptide_rt', 'peptide_detectability',
                                                 'peptide_mean', 'peptide_mean_diff']
experiment_params['Peptide'] = dict(
    encoding_type='Meta',
    features=features,
)
# Peptide+Protein 
features = [
    'len', 'gravy', 'helix', 'turn', 'sheet', 'aromaticity',
    'instability_index', 'isoelectric_point', 'molar_extinction_coefficient'
]
peptide_features = ['peptide_' + f for f in features] + ['peptide_rt', 'peptide_detectability',
                                                         'peptide_mean', 'peptide_mean_diff']
protein_features = ['protein_' + f for f in features]
features = peptide_features + protein_features
experiment_params['Peptide+Protein'] = dict(
    encoding_type='Meta',
    features=features,
)
# Counts+Peptide
features = [
    'len', 'gravy', 'helix', 'turn', 'sheet', 'aromaticity',
    'instability_index', 'isoelectric_point', 'molar_extinction_coefficient'
]
features = ['peptide_' + f for f in features] + ['peptide_rt', 'peptide_detectability',
                                                 'peptide_mean', 'peptide_mean_diff']
experiment_params['Counts+Peptide'] = dict(
    encoding_type='Counts',
    features=features,
)
# Counts+Peptide+Protein
features = [
    'len', 'gravy', 'helix', 'turn', 'sheet', 'aromaticity',
    'instability_index', 'isoelectric_point', 'molar_extinction_coefficient'
]
peptide_features = ['peptide_' + f for f in features] + ['peptide_rt', 'peptide_detectability',
                                                         'peptide_mean', 'peptide_mean_diff']
protein_features = ['protein_' + f for f in features]
features = peptide_features + protein_features
experiment_params['Counts+Peptide+Protein'] = dict(
    encoding_type='Counts',
    features=features,
)
# Important
features = [
    'peptide_len', 'peptide_mean', 'peptide_mean_diff',
    'peptide_detectability', 'peptide_rt', 
]
experiment_params['Important'] = dict(
    encoding_type='Counts',
    features=features,
)

# Models to test and exp to run
models_to_test = [
#     LinearRegression,
#     DummyRegressor,
#     KNeighborsRegressor,
#     Ridge,
#     ElasticNet,
    KernelRidge,
    RandomForestRegressor,
    MLPRegressor,
#     XGBRegressor,
]
exp_to_run = [
#     'Counts',
#     'Relative',
#     'Binary',
#     'Counts+len',
#     'Relative+len',
#     'Binary+len',
#     'Doc2Vec',
#     'Bert',
#     'Peptide',
#     'Peptide+Protein',
#     'Counts+Peptide',
#     'Counts+Peptide+Protein'
    'Important'
]

# Get only relevant exp and models
names = set([m.__name__ for m in models_to_test])
to_del = []
for k in optim_params:
    if k not in names:
        to_del.append(k)
for k in to_del:
    del optim_params[k]

experiment_params = {k:experiment_params[k] for k in exp_to_run}

## Optimization

In [10]:
container = ResultsContainer()
for exp_name, exp_params in experiment_params.items():
    xs, y, groups, _ = get_data(df, exp_params['features'])
    x = xs[exp_params['encoding_type']]
    kf = GroupKFold()
    print(exp_name)
    for train_index, test_index in kf.split(x, y, groups):
        # Get Data
        x_train, y_train, groups_train = x[train_index], y[train_index], groups[train_index]
        x_test, y_test = x[test_index], y[test_index]
        cv_params = {
            'scoring': 'neg_mean_squared_error',
            'cv': GroupKFold(),
            'groups': groups_train,
            'n_jobs': -1
        }
        # Run optimization
        params, history = optimize_models(x_train, y_train,
                                          optim_params, cv_params,
                                          evals=30)
        
        # Run model comparison
        results = compare_models(x_train, y_train, x_test, y_test,
                                 models_to_test, optim_params,
                                 params)
        # Store results
        container.update_from_list(exp_name, history, params, results)
container.update_file('Results/reproducibility_results.pkl')

Important
100%|███████████████████████████████████████████████| 30/30 [04:51<00:00,  9.72s/trial, best loss: 0.04955523055606753]


C:\Users\mosin\anaconda3\envs\proteins\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


## Feature Importances

In [6]:
# from sklearn.inspection import permutation_importance

# container = pickle.load(open('Results/reproducibility_results.pkl', 'rb'))
# results = pickle.load(open('Results/reproducibility_feature_importances.pkl', 'rb'))
# for exp_name, exp_params in experiment_params.items():
#     print('-'*5, exp_name, '-'*5)
#     xs, y, groups, feature_names = get_data(df, exp_params['features'])
#     x = xs[exp_params['encoding_type']]
#     feature_names = feature_names[-x.shape[1]:]
#     kf = GroupKFold()
    
#     results[exp_name] = {}
#     model_results = {}
#     for Model in models_to_test:
#         model_name = Model.__name__
#         print(model_name)
#         all_params = container[exp_name][model_name]['params']
#         params = []
#         for i in range(5):
#             p = {}
#             for k, v in all_params.items():
#                 p[k] = v[i]
#             params.append(p)

#         model_results = []
#         for i, (train_index, test_index) in enumerate(kf.split(x, y, groups)):
#             x_train, y_train, groups_train = x[train_index], y[train_index], groups[train_index]
#             x_test, y_test = x[test_index], y[test_index]

#             model = get_model(Model, optim_params, {model_name: params[i]})[1]
#             model.fit(x_train, y_train)
#             r = permutation_importance(model, x_test, y_test,
#                                        n_repeats=10,
#                                        scoring='explained_variance')['importances']
#             model_results.append(r)
#         model_results = np.hstack(model_results)
#         model_results = {f_name:list(vals) for f_name, vals in zip(feature_names, model_results)}
#         results[exp_name][model_name] = model_results
# pickle.dump(results, open('Results/reproducibility_feature_importances.pkl', 'wb'))

----- Counts -----
KernelRidge
RandomForestRegressor
MLPRegressor
----- Peptide -----
KernelRidge
RandomForestRegressor
MLPRegressor


C:\Users\mosin\anaconda3\envs\proteins\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


----- Counts+Peptide -----
KernelRidge
RandomForestRegressor
MLPRegressor


# Ensemble

In [7]:
def get_top_params(optim_history, n):
    idxs = np.argsort(optim_history['loss'])[:n]
    curr_params = []
    params = [{} for i in range(n)]
    for i, idx in enumerate(idxs):
        for k, v in optim_history.items():
            if k != 'loss':
                params[i][k] = v[idx]
    return params

def get_ensemble(models_to_test, models_params, optim_params):
    models = []
    for Model in models_to_test:
        model_name = Model.__name__
        for i, p in enumerate(models_params[model_name]):
            models.append((model_name+str(i+1), get_model(Model, optim_params, {model_name: p})[1]))
    return StackingRegressor(models)

In [8]:
# container = pickle.load(open('Results/reproducibility_results.pkl', 'rb'))
# # results = pickle.load(open('Results/reproducibility_ensemble.pkl', 'rb'))

# results = {}
# for exp_name, exp_params in experiment_params.items():
#     print('-'*5, exp_name, '-'*5)
#     xs, y, groups, feature_names = get_data(df, exp_params['features'])
#     x = xs[exp_params['encoding_type']]
#     feature_names = feature_names[-x.shape[1]:]
#     kf = GroupKFold()
    
#     exp_results = defaultdict(list)
#     for fold, (train_index, test_index) in enumerate(kf.split(x, y, groups)):
#         print('Fold:', fold+1)
#         x_train, y_train, groups_train = x[train_index], y[train_index], groups[train_index]
#         x_test, y_test = x[test_index], y[test_index]
        
#         models_params = {}
#         for Model in models_to_test:
#             model_name = Model.__name__
#             optim_history = container[exp_name][model_name]['optim_history'][fold]
#             models_params[model_name] = get_top_params(optim_history, 2)
        
#         model = get_ensemble(models_to_test, models_params, optim_params)
#         preds_train = model.fit(x_train, y_train).predict(x_train)
#         preds_test = model.predict(x_test)
        
#         metrics = dict(
#             MSE=mean_squared_error,
#             MAE=mean_absolute_error,
#             R2=r2_score,
#             EV=explained_variance_score
#         )
#         for metric, func in metrics.items():
#             exp_results['Train '+metric].append(func(y_train, preds_train))
#             exp_results['Test '+metric].append(func(y_test, preds_test))
#     results[exp_name] = dict(exp_results)

# pickle.dump(results, open('Results/reproducibility_ensemble.pkl', 'wb'))